In [1]:
import torch
import torchvision
import torchvision.transforms.functional as F
from PIL import Image
import time
import os
import shutil
import cv2
import numpy as np
import pandas as pd
from PIL import Image
from glob import glob
from matplotlib import pyplot as plt
import matplotlib
%matplotlib inline
from time import gmtime, strftime
import time
from datetime import datetime

import os


device = 'cuda:0'

C:\Users\s.ezati\Anaconda3\envs\emty_table\lib\site-packages\tqdm\auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
dict = {
}
dict["172"]=""
dict

{'172': ''}

In [8]:
for key, value in dict.items():
    # do something with value
    if key == "172":
        dict[key] = "5"

In [10]:
count = 0
for i in range(0,10):
    dict["172"]= count
    print(dict)
    count = count+1

{'172': 0}
{'172': 1}
{'172': 2}
{'172': 3}
{'172': 4}
{'172': 5}
{'172': 6}
{'172': 7}
{'172': 8}
{'172': 9}


In [2]:
frame_size = 1800
frame_ratio = 0.5625

# <font color= LimeGreen> Model </font>


In [3]:
model = torch.hub.load('ultralytics/yolov5', 'custom', path='yolov5x6.pt',device='cuda:0')

Using cache found in C:\Users\s.ezati/.cache\torch\hub\ultralytics_yolov5_master
YOLOv5  2023-1-21 Python-3.10.9 torch-1.13.0+cu117 CUDA:0 (NVIDIA GeForce RTX 3060, 12287MiB)

Fusing layers... 
YOLOv5x6 summary: 574 layers, 140730220 parameters, 0 gradients
Adding AutoShape... 


In [4]:
def Overlap_Percent(boxA, boxB):
    xA = max(boxA[0], boxB[0])
    yA = max(boxA[1], boxB[1])
    xB = min(boxA[2], boxB[2])
    yB = min(boxA[3], boxB[3])

    interArea = abs(max((xB - xA, 0)) * max((yB - yA), 0))
    if interArea == 0:
        return 0

    boxAArea = abs((boxA[2] - boxA[0]) * (boxA[3] - boxA[1]))
    boxBArea = abs((boxB[2] - boxB[0]) * (boxB[3] - boxB[1]))

    iou = interArea / float(boxAArea + boxBArea - interArea)
    iou = iou * 100
    
    return iou



In [5]:
def Object_Detection(model,person_confidence_tresh, img):
    
    img = Image.fromarray(img)
    img = F.adjust_brightness(img, 0.95)
    img = F.adjust_contrast(img,1)
    img = F.adjust_sharpness(img,0.6) 
#     cv2.imwrite(f'dataset/frame0101.jpg', np.array(img))
#     img = cv2.cvtColor(np.array(img),cv2.COLOR_RGB2BGR)   
#     cv2.imwrite(f'dataset/frame000.jpg', img)
    img = np.array(img)
    chair_cordination = []
    person_cordination = []

    with torch.no_grad(): results = model(np.array(img))

    if len(results.xyxy[0]) == 0: pass
    else:        
        for indx,res in enumerate(results.xyxy[0]):
            predict = results.pandas().xyxy[0]['name'][indx]
#             print(predict)
            if predict == "chair":
                res = np.array(res.detach().cpu())
                if res[4]*100 > 40:
                    x = round(res[0])
                    y = round(res[1])
                    w = round(res[2])
                    h = round(res[3])
                    chair_cordination.append([x,y,w,h])
            elif predict == "person":
                res = np.array(res.detach().cpu())
                if res[4]*100 > person_confidence_tresh:
                    x = round(res[0])
                    y = round(res[1])
                    w = round(res[2])
                    h = round(res[3])
                    person_cordination.append([x,y,w,h])
            else:pass
    
    return chair_cordination,person_cordination
        

In [6]:
def Employee_Status(Object_Detection, img , table_cordination):
    area = (table_cordination[2] * table_cordination[3]) / 100
    if area >= 2000:
        dist_tresh = ((int(table_cordination[0] + table_cordination[2]) /
                       int(table_cordination[1] + table_cordination[3])) *
                      100 + 10 * 2)#2.75
#         print(dist_tresh)
        person_confidence_tresh = 30
    elif 1300 <= area < 2000:
        dist_tresh = ((int(table_cordination[0] + table_cordination[2]) /
                           int(table_cordination[1] + table_cordination[3])) *
                          100 + 10 * 3.25)#2.75
        person_confidence_tresh = 30
    elif 800 <= area < 1300:
        dist_tresh = ((int(table_cordination[0] + table_cordination[2]) /
                           int(table_cordination[1] + table_cordination[3])) *
                          100 + 10 * (-3.25))#2.75
        person_confidence_tresh = 30
    elif 150 <= area < 800:
        dist_tresh = ((int(table_cordination[0] + table_cordination[2]) /
                       int(table_cordination[1] + table_cordination[3])) *
                      100 + 10 * 6.15) #0.75
        person_confidence_tresh = 30
        print(dist_tresh)

    else:
        dist_tresh = ((int(table_cordination[0] + table_cordination[2]) /
                       int(table_cordination[1] + table_cordination[3])) * 
                      100 + 10 * 3)
        person_confidence_tresh = 40
#         print(dist_tresh)

    img_ = img[table_cordination[1]:table_cordination[3]+table_cordination[1], table_cordination[0]:table_cordination[2]+table_cordination[0]]
    cv2.imwrite(f'dataset/frame.jpg', img_)
    chair_cordination, person_cordination  = Object_Detection(model,person_confidence_tresh,img_)
    
    overlap_list = []
    person_list = []
    chair_list = []
    
    color = (0, 0, 255)
    status = ""
    
    if person_cordination:
        if chair_cordination:
            for person in person_cordination:
                for chair in chair_cordination:
                    coverage = Overlap_Percent(person, chair)    
                    overlap_list.append(coverage) 
                    person_list.append(person)
                    chair_list.append(chair)
        
            if max(overlap_list) < 15:
                status = "person is not at table :( "
                color = (255,0,0)
#                 print("1: maxoverlap_list:", max(overlap_list))
                
            else:
#                 print("2 : max(overlap_list:", max(overlap_list))
                index = overlap_list.index(max(overlap_list))
                person = person_list[index]
                chair = chair_list[index]
                
                if person[1] > chair[1]:
                    status = "person is working :) "   
                    color = (178,255,102)
                else:
                    dist_y = np.linalg.norm(person[1] - chair[1])
                    print(dist_y)
#                     print(dist_tresh)
                    if dist_y >= dist_tresh:
                        status = "person is at table without sitting on chair -_- " 
                        color = (0, 255, 255)
                        
                    else:
                        status = "person is working :) "
                        color = (178, 255, 102)
                        
        else: 
            status = "chair is not detected" 
            color = (255, 128, 0)
    else:
        status = "person is not at table :( "
        color = (255, 0, 0)
    
    
    
        
    return status , color , img_
    


In [7]:
def show_frame(img, status = "not set" ,time_list = ['0:00:00.00','0:00:00.00','0:00:00.00','0:00:00.00'],report_time = '0' ,ROI = [0,0,0,0] , employee_id = 'not define', color = (255,255,255)):
    
    img_original = cv2.cvtColor(np.array(img),cv2.COLOR_BGR2RGB)
    report_time = str(report_time)
    current_time_list = [pd.Timedelta('0:00:00.00'),pd.Timedelta('0:00:00.00'),pd.Timedelta('0:00:00.00'),pd.Timedelta('0:00:00.00')]
    lines = []
    
    if status == "person is working :) ": 
        if (os.path.exists(f"database/{employee_id}/person_is_working.txt") == False):
            f = open(f"database/{employee_id}/person_is_working.txt", "a")
        f = open(f"database/{employee_id}/person_is_working.txt", "a")
        f.write(report_time)
        f.write('\n')
        f.close()
        
    if status == "person is at table without sitting on chair -_- ": 
        if (os.path.exists(f"database/{employee_id}/person_is_at_table_without_sitting_on_chair.txt") == False):
            f = open(f"database/{employee_id}/person_is_at_table_without_sitting_on_chair.txt", "a")
            
        f = open(f"database/{employee_id}/person_is_at_table_without_sitting_on_chair.txt", "a")
        f.write(report_time)
        f.write('\n')
        f.close()
        
    if status == "chair is not detected": 
        if (os.path.exists(f"database/{employee_id}/chair_is_not_detected.txt") == False):
            f = open(f"database/{employee_id}/chair_is_not_detected.txt", "a")
        f = open(f"database/{employee_id}/chair_is_not_detected.txt", "a")
        f.write(report_time)
        f.write('\n')
        f.close()
        
    if status == "person is not at table :( ": 
        if (os.path.exists(f"database/{employee_id}/person_is_not_at_table.txt") == False):
            f = open(f"database/{employee_id}/person_is_not_at_table.txt" , "a")
        f = open(f"database/{employee_id}/person_is_not_at_table.txt", "a")
        f.write(report_time)
        f.write('\n')
        f.close()
        
     
    if status == "person is working :) ": 
        f = open(f'database/{employee_id}/person_is_working.txt', 'r')
        Lines = f.readlines()
        f.close()
        for line in Lines:
            line = line.removesuffix("\n")
            current_time_list[0] = current_time_list[0] + pd.Timedelta(line)
            
    if status == "person is at table without sitting on chair -_- ": 
        f = open(f'database/{employee_id}/person_is_at_table_without_sitting_on_chair.txt', 'r')
        Lines = f.readlines()
        f.close()
        for line in Lines:
            line = line.removesuffix("\n")
            current_time_list[1] = current_time_list[1] + pd.Timedelta(line)
            
    if status == "chair is not detected": 
        f = open(f'database/{employee_id}/chair_is_not_detected.txt', 'r')
        Lines = f.readlines()
        f.close()
        for line in Lines:
            line = line.removesuffix("\n")
            current_time_list[2] = current_time_list[2] + pd.Timedelta(line)
            
    if status == "person is not at table :( ": 
        f = open(f'database/{employee_id}/person_is_not_at_table.txt', 'r')
        Lines = f.readlines()
        f.close()
        for line in Lines:
            line = line.removesuffix("\n")
            current_time_list[3] = current_time_list[3] + pd.Timedelta(line)

    
    img_original= cv2.rectangle(np.array(img_original), 
                                (ROI[0]-10,ROI[1]-110,
                                 ROI[2]+10,110), 
                                 (0,0,100), cv2.FILLED, 1)
    
    time_text_list = [f"working time: {str(str(time_list[0]).split(' ')[2]).split('.')[0]} ",
                 f"person is at table without",f"sitting on chair: {str(str(time_list[1]).split(' ')[2]).split('.')[0]}",
                 f"chair is not detected: {str(str(time_list[2]).split(' ')[2]).split('.')[0]}",
                 f"person is not at table: {str(str(time_list[3]).split(' ')[2]).split('.')[0]}"]
    time_color_list = [(178,255,102),(0,255,255),(0,255,255),(255,128,0),(255,0,0)]
    count = 0
    for l,c in zip(time_text_list,time_color_list):
        img_original = cv2.putText(np.array(img_original),
                           l,
                           (ROI[0]-10,ROI[1]-90 + count), 
                            fontFace=cv2.FONT_HERSHEY_TRIPLEX, 
                            fontScale=0.4,color = c, thickness=1)
        count = count + 20
        
    
    img_original= cv2.rectangle(np.array(img_original), 
                                (ROI[0],ROI[1],
                                 ROI[2],ROI[3]), 
                                 color, 1)

    img_original= cv2.rectangle(np.array(img_original), 
                                (ROI[0],ROI[1]+ROI[3]-25,
                                  ROI[2],50), 
                                 color, cv2.FILLED, 1)
    
    
    status_text_list = [f"person id : {employee_id} ",f"status : {status}"]
    s_count = 0
    for stl in status_text_list:
        img_original = cv2.putText(np.array(img_original),
                                   stl, 
                                   (ROI[0], ROI[1]+ROI[3]-10 + s_count), 
                                    fontFace=cv2.FONT_HERSHEY_TRIPLEX, 
                                    fontScale=0.4,color = (0,0,0), thickness=1)
        s_count = s_count + 20

    img_original = cv2.resize(img_original, (frame_size, round(frame_size * frame_ratio)))
    img_original = cv2.cvtColor(np.array(img_original),cv2.COLOR_RGB2BGR)   
    
    return img_original , current_time_list

In [8]:
def mouse_click(event, x, y, flags, param):
    global ROI_Region
    global img_original
    global filter_ 

    if event == cv2.EVENT_RBUTTONDOWN:
        ROI_Region = cv2.selectROI("Select Area",img_original)
        print(ROI_Region)
        cv2.destroyAllWindows()
    elif event == cv2.EVENT_MBUTTONDOWN: 
        filter_ = ''
        ROI_Region = [0,0,0,0]
    else: pass

In [9]:
from threading import Thread

class ThreadedCamera(object):
    def __init__(self, source='cam:Sayda@0.0.0.0:88/videoMain'):

        self.capture = cv2.VideoCapture(f'rtsp://{source}')

        self.thread = Thread(target = self.update, args = ())
        self.thread.daemon = True
        self.thread.start()

        self.status = False
        self.frame  = None

    def update(self):
        while True:
            if self.capture.isOpened(): (self.status, self.frame) = self.capture.read()
            else: continue

    def grab_frame(self):
        if self.status: return self.frame
        else: return None 
    
    def end_frame(self):
        self.capture.release()

In [10]:
from threading import Thread

class ThreadedVideo(object):
    def __init__(self, source='cam:Sayda@0.0.0.0:88/videoMain'):

        self.capture = cv2.VideoCapture(source)
        self.capture.set(cv2.CAP_PROP_BUFFERSIZE, 2)

        self.thread = Thread(target = self.update, args = ())
        self.thread.daemon = True
        self.thread.start()

        self.status = False
        self.frame  = None

    def update(self):
        while True:
            if self.capture.isOpened(): (self.status, self.frame) = self.capture.read()
            else: continue

    def grab_frame(self):
        if self.status: return self.frame
        else: return None 
    
    def end_frame(self):
        self.capture.release()

In [11]:
streamer_121 = ThreadedCamera('rtsp:Sayda@0.0.0.0')


#_________________________________________________________________________________________

person_cordination_dict_121 = {
    "512320" : [159-50, 286, 180+35, 306],
    "512313" : [339+20, 559-40, 572, 399]
   
    
}

person_time_list_121 = {
    "512320" : ['0 day 0:00:00.00','0 day 0:00:00.00','0 day 0:00:00.00','0 day 0:00:00.00'],
    "512313" : ['0 day 0:00:00.00','0 day 0:00:00.00','0 day 0:00:00.00','0 day 0:00:00.00']
}

In [12]:

        
for key in person_cordination_dict_121:
    if not os.path.isdir(f'database/{key}'):
        os.makedirs(f'database/{key}')

In [ ]:

while True: 

    img_original_121 = streamer_121.grab_frame()

    
    try: img_original_121 = Image.fromarray(img_original_121)
    except AttributeError: continue
       
    img_original_121 = cv2.resize(np.array(img_original_121), (frame_size, round(frame_size * frame_ratio)))

    img_121 = img_original_121.copy()

    
  
    if img_original_121 is None: 
        img_res_121 = np.zeros([round(frame_size * frame_ratio), frame_size,3],dtype=np.uint8)
    else:
        img_show_121 = img_original_121
        for key in person_cordination_dict_121:
            start_time_121 = datetime.now()
            status_121, color_121 , img_121_ = Employee_Status(Object_Detection, img_121, 
                                                                        person_cordination_dict_121[key])
            end_time_121 = datetime.now()
            report_time_121 = end_time_121 - start_time_121
            img_res_121 , current_time_list_121 = show_frame(img_show_121 , time_list = person_time_list_121[key], status = status_121,
                                                     report_time = report_time_121 , ROI = person_cordination_dict_121[key]  
                                                     , employee_id = key, color = color_121)
            img_show_121 = img_res_121
            person_time_list_121[key][0] = (person_time_list_121[key][0] + current_time_list_121[0]) 
            person_time_list_121[key][1] = (person_time_list_121[key][1] + current_time_list_121[1]) 
            person_time_list_121[key][2] = (person_time_list_121[key][2] + current_time_list_121[2]) 
            person_time_list_121[key][3] = (person_time_list_121[key][3] + current_time_list_121[3]) 


    
    img_res_121 = cv2.resize(img_show_121, (1680, 1050))
    
    #__________________________________________________________________________

    cv2.imshow('Empty Table Detection', np.array(img_res_121))
    #__________________________________________________________________________

    if cv2.waitKey(1) & 0xFF == ord('q'): 
        
        cv2.destroyAllWindows()
        streamer_121.end_frame()
        
        for key in person_cordination_dict_121:
            if(os.path.exists(f"database/{key}/person_is_working.txt") == True):
                os.remove(f"database/{key}/person_is_working.txt")
                    
            if(os.path.exists(f"database/{key}/person_is_at_table_without_sitting_on_chair.txt") == True):
                os.remove(f"database/{key}/person_is_at_table_without_sitting_on_chair.txt")
                    
            if(os.path.exists(f"database/{key}/chair_is_not_detected.txt") == True):
                os.remove(f"database/{key}/chair_is_not_detected.txt")
                    
            if(os.path.exists(f"database/{key}/person_is_not_at_table.txt") == True):
                os.remove(f"database/{key}/person_is_not_at_table.txt")
                
        break 



116.22972972972973
76.0
116.22972972972973
116.22972972972973
116.22972972972973
116.22972972972973
116.22972972972973
116.22972972972973
116.22972972972973
116.22972972972973
116.22972972972973
116.22972972972973
116.22972972972973
116.22972972972973
116.22972972972973
116.22972972972973
116.22972972972973
116.22972972972973
116.22972972972973
116.22972972972973
116.22972972972973
116.22972972972973
116.22972972972973
116.22972972972973
116.22972972972973
116.22972972972973
116.22972972972973
116.22972972972973
116.22972972972973
116.22972972972973
116.22972972972973
116.22972972972973
116.22972972972973
116.22972972972973
116.22972972972973
116.22972972972973
116.22972972972973
116.22972972972973
116.22972972972973
116.22972972972973
116.22972972972973
116.22972972972973
116.22972972972973
116.22972972972973
116.22972972972973
116.22972972972973
116.22972972972973
116.22972972972973
116.22972972972973
116.22972972972973
116.22972972972973
116.22972972972973
116.22972972972973
116.229

116.22972972972973
116.22972972972973
116.22972972972973
116.22972972972973
116.22972972972973
116.22972972972973
116.22972972972973
116.22972972972973
116.22972972972973
116.22972972972973
116.22972972972973
116.22972972972973
116.22972972972973
116.22972972972973
116.22972972972973
116.22972972972973
116.22972972972973
116.22972972972973
116.22972972972973
116.22972972972973
116.22972972972973
116.22972972972973
116.22972972972973
116.22972972972973
116.22972972972973
116.22972972972973
116.22972972972973
116.22972972972973
116.22972972972973
116.22972972972973
116.22972972972973
116.22972972972973
116.22972972972973
116.22972972972973
116.22972972972973
116.22972972972973
116.22972972972973
64.0
116.22972972972973
116.22972972972973
116.22972972972973
116.22972972972973
116.22972972972973
116.22972972972973
116.22972972972973
116.22972972972973
116.22972972972973
116.22972972972973
116.22972972972973
116.22972972972973
116.22972972972973
116.22972972972973
116.22972972972973
116.229

116.22972972972973
116.22972972972973
116.22972972972973
116.22972972972973
116.22972972972973
116.22972972972973
116.22972972972973
116.22972972972973
116.22972972972973
116.22972972972973
116.22972972972973
116.22972972972973
116.22972972972973
116.22972972972973
116.22972972972973
116.22972972972973
116.22972972972973
116.22972972972973
116.22972972972973
116.22972972972973
44.0
116.22972972972973
44.0
116.22972972972973
44.0
116.22972972972973
44.0
116.22972972972973
44.0
116.22972972972973
44.0
116.22972972972973
44.0
116.22972972972973
44.0
116.22972972972973
44.0
116.22972972972973
44.0
116.22972972972973
44.0
116.22972972972973
43.0
116.22972972972973
44.0
116.22972972972973
42.0
116.22972972972973
43.0
116.22972972972973
44.0
116.22972972972973
116.22972972972973
116.22972972972973
116.22972972972973
44.0
116.22972972972973
43.0
116.22972972972973
116.22972972972973
116.22972972972973
116.22972972972973
116.22972972972973
116.22972972972973
116.22972972972973
116.2297297297297

60.0
116.22972972972973
116.22972972972973
56.0
116.22972972972973
54.0
116.22972972972973
56.0
116.22972972972973
59.0
116.22972972972973
56.0
116.22972972972973
57.0
116.22972972972973
58.0
116.22972972972973
60.0
116.22972972972973
60.0
116.22972972972973
116.22972972972973
116.22972972972973
116.22972972972973
116.22972972972973
116.22972972972973
116.22972972972973
116.22972972972973
116.22972972972973
116.22972972972973
116.22972972972973
116.22972972972973
116.22972972972973
116.22972972972973
116.22972972972973
116.22972972972973
116.22972972972973
116.22972972972973
116.22972972972973
116.22972972972973
116.22972972972973
116.22972972972973
116.22972972972973
116.22972972972973
116.22972972972973
116.22972972972973
116.22972972972973
116.22972972972973
116.22972972972973
116.22972972972973
116.22972972972973
116.22972972972973
116.22972972972973
116.22972972972973
116.22972972972973
116.22972972972973
116.22972972972973
116.22972972972973
116.22972972972973
116.22972972972973


116.22972972972973
116.22972972972973
2.0
116.22972972972973
116.22972972972973
116.22972972972973
116.22972972972973
116.22972972972973
116.22972972972973
116.22972972972973
116.22972972972973
116.22972972972973
116.22972972972973
116.22972972972973
116.22972972972973
116.22972972972973
116.22972972972973
116.22972972972973
116.22972972972973
116.22972972972973
116.22972972972973
116.22972972972973
116.22972972972973
116.22972972972973
116.22972972972973
116.22972972972973
116.22972972972973
116.22972972972973
0.0
116.22972972972973
0.0
116.22972972972973
0.0
116.22972972972973
116.22972972972973
116.22972972972973
116.22972972972973
116.22972972972973
116.22972972972973
116.22972972972973
116.22972972972973
116.22972972972973
116.22972972972973
116.22972972972973
116.22972972972973
116.22972972972973
0.0
116.22972972972973
116.22972972972973
116.22972972972973
116.22972972972973
116.22972972972973
116.22972972972973
116.22972972972973
116.22972972972973
1.0
116.22972972972973
116.229

...

116.22972972972973
99.0
116.22972972972973
98.0
116.22972972972973
99.0
116.22972972972973
99.0
116.22972972972973
98.0
116.22972972972973
99.0
116.22972972972973
98.0
116.22972972972973
98.0
116.22972972972973
98.0
116.22972972972973
98.0
116.22972972972973
98.0
116.22972972972973
98.0
116.22972972972973
98.0
116.22972972972973
98.0
116.22972972972973
99.0
116.22972972972973
99.0
116.22972972972973
98.0
116.22972972972973
99.0
116.22972972972973
99.0
116.22972972972973
102.0
116.22972972972973
103.0
116.22972972972973
103.0
116.22972972972973
103.0
116.22972972972973
104.0
116.22972972972973
104.0
116.22972972972973
101.0
116.22972972972973
101.0
116.22972972972973
99.0
116.22972972972973
110.0
116.22972972972973
112.0
116.22972972972973
119.0
116.22972972972973
120.0
116.22972972972973
120.0
116.22972972972973
111.0
116.22972972972973
98.0
116.22972972972973
98.0
116.22972972972973
100.0
116.22972972972973
99.0
116.22972972972973
98.0
116.22972972972973
98.0
116.22972972972973
99.0
1

116.22972972972973
111.0
116.22972972972973
110.0
116.22972972972973
116.22972972972973
116.22972972972973
144.0
116.22972972972973
112.0
116.22972972972973
112.0
116.22972972972973
113.0
112.0
116.22972972972973
112.0
116.22972972972973
112.0
116.22972972972973
113.0
116.22972972972973
114.0
116.22972972972973
111.0
116.22972972972973
111.0
116.22972972972973
111.0
116.22972972972973
111.0
116.22972972972973
112.0
116.22972972972973
109.0
116.22972972972973
109.0
116.22972972972973
109.0
116.22972972972973
109.0
116.22972972972973
109.0
116.22972972972973
109.0
116.22972972972973
109.0
116.22972972972973
109.0
116.22972972972973
110.0
116.22972972972973
109.0
116.22972972972973
110.0
116.22972972972973
110.0
116.22972972972973
110.0
116.22972972972973
113.0
116.22972972972973
153.0
116.22972972972973
116.22972972972973
116.22972972972973
114.0
116.22972972972973
110.0
116.22972972972973
110.0
116.22972972972973
110.0
116.22972972972973
110.0
116.22972972972973
108.0
116.22972972972973

116.22972972972973
106.0
116.22972972972973
107.0
116.22972972972973
105.0
116.22972972972973
102.0
116.22972972972973
105.0
116.22972972972973
96.0
116.22972972972973
98.0
116.22972972972973
104.0
116.22972972972973
104.0
116.22972972972973
102.0
116.22972972972973
104.0
116.22972972972973
103.0
116.22972972972973
103.0
116.22972972972973
103.0
116.22972972972973
103.0
116.22972972972973
103.0
116.22972972972973
103.0
116.22972972972973
103.0
116.22972972972973
103.0
116.22972972972973
103.0
116.22972972972973
103.0
116.22972972972973
103.0
116.22972972972973
103.0
116.22972972972973
103.0
116.22972972972973
101.0
116.22972972972973
101.0
116.22972972972973
101.0
116.22972972972973
101.0
116.22972972972973
100.0
116.22972972972973
102.0
116.22972972972973
102.0
116.22972972972973
103.0
116.22972972972973
103.0
116.22972972972973
102.0
116.22972972972973
100.0
116.22972972972973
104.0
116.22972972972973
104.0
116.22972972972973
104.0
116.22972972972973
107.0
116.22972972972973
107.0
11

116.22972972972973
92.0
116.22972972972973
92.0
116.22972972972973
89.0
116.22972972972973
89.0
116.22972972972973
89.0
116.22972972972973
116.0
116.22972972972973
116.22972972972973
116.22972972972973
42.0
116.22972972972973
46.0
116.22972972972973
88.0
116.22972972972973
88.0
116.22972972972973
82.0
116.22972972972973
88.0
116.22972972972973
89.0
116.22972972972973
90.0
116.22972972972973
89.0
116.22972972972973
89.0
116.22972972972973
89.0
116.22972972972973
88.0
116.22972972972973
87.0
116.22972972972973
83.0
116.22972972972973
85.0
116.22972972972973
86.0
116.22972972972973
85.0
116.22972972972973
86.0
116.22972972972973
89.0
116.22972972972973
87.0
116.22972972972973
90.0
116.22972972972973
52.0
116.22972972972973
49.0
116.22972972972973
86.0
116.22972972972973
90.0
116.22972972972973
90.0
116.22972972972973
89.0
116.22972972972973
91.0
116.22972972972973
90.0
116.22972972972973
91.0
116.22972972972973
56.0
116.22972972972973
92.0
116.22972972972973
89.0
116.22972972972973
88.0
1

116.22972972972973
83.0
116.22972972972973
83.0
116.22972972972973
84.0
116.22972972972973
84.0
116.22972972972973
84.0
116.22972972972973
84.0
116.22972972972973
84.0
116.22972972972973
83.0
116.22972972972973
84.0
116.22972972972973
85.0
116.22972972972973
85.0
116.22972972972973
84.0
116.22972972972973
84.0
116.22972972972973
83.0
